# Proyecto 1 – Limpieza (Archivo NUEVO)
**Fuente**: `/mnt/data/establecimientos_diversificado_raw_concat.csv`

Este cuaderno realiza **exclusivamente limpieza** y deja la trazabilidad de cada paso.
Incluye una regla específica para eliminar **bullets iniciales** en `ESTABLECIMIENTO`/`DIRECCION` (p. ej., `- IGA`, `-IGA`) sin eliminar guiones internos.

## Pasos
1. Carga y normalización de **nombres de columnas** (a MAYÚSCULAS)
2. Limpieza general (espacios/guiones/vacíos)
3. Remoción de bullets iniciales y placeholders
4. Normalización de `ESTABLECIMIENTO`, `DIRECCION`, `TELEFONO`
5. Estandarización de categóricas
6. Clave canónica y **marcado** de posibles duplicados
7. Guardado de salidas (CSV limpio, duplicados, bitácora)


In [ ]:
import re, unicodedata, pandas as pd
from collections import OrderedDict
from datetime import datetime
RAW_PATH = '/mnt/data/establecimientos_diversificado_raw_concat.csv'
df = pd.read_csv(RAW_PATH, dtype=str)
df.columns = [c.upper() for c in df.columns]
df.shape

## Utilidades de normalización

In [ ]:
def strip_accents(s: str):
    import unicodedata
    if not isinstance(s, str): return s
    return ''.join(ch for ch in unicodedata.normalize('NFKD', s) if not unicodedata.combining(ch))

import re
def collapse_spaces_and_hyphens(s: str):
    if not isinstance(s, str): return s
    s = s.replace('—', '-').replace('–', '-').replace('−', '-').replace('­', '')
    s = re.sub(r'[ \t\u00A0]+', ' ', s)
    s = re.sub(r'-{2,}', '-', s)
    s = re.sub(r'\s*-\s*', ' - ', s)
    s = re.sub(r'\s{2,}', ' ', s)
    return s.strip()

def normalize_text_basic(s: str):
    if not isinstance(s, str): return s
    s = s.strip().replace('"', '').replace('“','').replace('”','').replace('´', "'")
    s = strip_accents(s).upper()
    s = collapse_spaces_and_hyphens(s)
    return s

def remove_leading_bullets(s: str):
    if not isinstance(s, str): return s
    return re.sub(r'^\s*[-•]+\s*', '', s)

from collections import OrderedDict
def expand_address_abbrev(s: str):
    if not isinstance(s, str): return s
    txt = ' ' + s + ' '
    rules = OrderedDict([
        (r'\bAV[\.]?\b', ' AVENIDA '),
        (r'\bAVE[\.]?\b', ' AVENIDA '),
        (r'\bAVDA[\.]?\b', ' AVENIDA '),
        (r'\bBLVD[\.]?\b', ' BOULEVARD '),
        (r'\bCALZ[\.]?\b', ' CALZADA '),
        (r'\bCOL[\.]?\b', ' COLONIA '),
        (r'\bCOND[\.]?\b', ' CONDOMINIO '),
        (r'\bRES[\.]?\b', ' RESIDENCIAL '),
        (r'\bZ[\.]?\b(?=\s*\d)', ' ZONA '),
        (r'\bZONA\b\s*(?=\d)', ' ZONA '),
        (r'\bKM[\.]?\b', ' KM '),
        (r'\bNO[\.]?\b', ' NUMERO '),
        (r'\bN[\.]?\b(?=\s*\d)', ' NUMERO '),
        (r'\b#\b', ' NUMERO '),
        (r'\b2DA\b', ' 2A '),
        (r'\b3RA\b', ' 3A '),
    ])
    for pat, rep in rules.items():
        txt = re.sub(pat, rep, txt, flags=re.IGNORECASE)
    txt = collapse_spaces_and_hyphens(txt)
    return txt

def normalize_name(s: str):
    if not isinstance(s, str): return s
    s = remove_leading_bullets(s)
    s = normalize_text_basic(s)
    return s

def normalize_address(s: str):
    if not isinstance(s, str): return s
    s = remove_leading_bullets(s)
    s = normalize_text_basic(s)
    s = expand_address_abbrev(s)
    s = re.sub(r'\b(\d+)\s*\.\s*', r'\1 ', s)
    s = re.sub(r'\bKM\.\b', ' KM ', s)
    s = collapse_spaces_and_hyphens(s)
    return s

def clean_phone_field(s: str):
    if not isinstance(s, str): return None
    nums = re.findall(r'\d{8}', s)
    nums = [n for n in nums if not re.fullmatch(r'0{8}', n)]
    seen, unique = set(), []
    for n in nums:
        if n not in seen:
            seen.add(n); unique.append(n)
    return ' / '.join(unique) if unique else None

def make_duplicate_key(nombre, direccion, municipio, departamento) -> str:
    nombre_n = normalize_name(nombre) if isinstance(nombre, str) else ''
    direccion_n = normalize_address(direccion) if isinstance(direccion, str) else ''
    municipio_n = normalize_text_basic(municipio) if isinstance(municipio, str) else ''
    depto_n = normalize_text_basic(departamento) if isinstance(departamento, str) else ''
    key = f"{nombre_n} | {direccion_n} | {municipio_n} | {depto_n}"
    key = collapse_spaces_and_hyphens(key)
    return key


## Pipeline de limpieza

In [ ]:
bitacora = []
def log_step(accion: str, justificacion: str, resumen=None):
    bitacora.append({'timestamp': datetime.now().isoformat(timespec='seconds'),
                    'accion': accion, 'justificacion': justificacion, 'resumen': resumen or {}})

# 1) Limpieza general
obj_cols = [c for c in df.columns if df[c].dtype == 'object']
df[obj_cols] = df[obj_cols].apply(lambda s: s.map(lambda x: collapse_spaces_and_hyphens(x) if isinstance(x, str) else x))
df[obj_cols] = df[obj_cols].replace({'': pd.NA, 'nan': pd.NA, 'None': pd.NA, '-': pd.NA, '--': pd.NA, '—': pd.NA})
log_step('normalizar_basico', 'Colapsar espacios/guiones y estandarizar vacíos/guiones simples a NA.')

# 2) Backups
for col in ['ESTABLECIMIENTO','DIRECCION','TELEFONO']:
    if col in df.columns and f'{col}_ORIG' not in df.columns:
        df[f'{col}_ORIG'] = df[col]

# 3) Bullets/placeholders
for col in ['ESTABLECIMIENTO','DIRECCION']:
    if col in df.columns:
        df[col] = df[col].map(remove_leading_bullets)
import re
placeholders = re.compile(r'^[-–—]{1,}$')
for col in ['ESTABLECIMIENTO','DIRECCION','TELEFONO','MUNICIPIO','DEPARTAMENTO']:
    if col in df.columns:
        df.loc[df[col].astype(str).str.fullmatch(placeholders, na=False), col] = pd.NA
log_step('bullets_y_placeholders', 'Eliminar bullets iniciales y placeholders de guiones.')

# 4) Normalización campos clave
if 'ESTABLECIMIENTO' in df.columns:
    df['ESTABLECIMIENTO'] = df['ESTABLECIMIENTO'].map(lambda x: normalize_name(x) if isinstance(x, str) else x)
if 'DIRECCION' in df.columns:
    df['DIRECCION'] = df['DIRECCION'].map(lambda x: normalize_address(x) if isinstance(x, str) else x)
if 'TELEFONO' in df.columns:
    df['TELEFONO'] = df['TELEFONO'].map(lambda x: clean_phone_field(x) if isinstance(x, str) else x)
    df['TELEFONO_VALIDO'] = df['TELEFONO'].apply(lambda x: bool(x) if pd.notna(x) else False)
log_step('normalizar_campos_clave', 'Nombre/dirección normalizados; extracción de teléfonos válidos.')

# 5) Categóricas
for col in ['DEPARTAMENTO','MUNICIPIO','SECTOR','AREA','STATUS','MODALIDAD','JORNADA','PLAN','NIVEL']:
    if col in df.columns:
        df[col] = df[col].map(lambda x: normalize_text_basic(x) if isinstance(x, str) else x)
log_step('categoricas', 'Estandarizar categóricas a MAYÚSCULAS sin acentos.')

# 6) Duplicados
for required in ['ESTABLECIMIENTO','DIRECCION','MUNICIPIO','DEPARTAMENTO']:
    if required not in df.columns:
        df[required] = pd.NA
df['CLAVE_DUP'] = df.apply(lambda r: make_duplicate_key(r['ESTABLECIMIENTO'], r['DIRECCION'], r['MUNICIPIO'], r['DEPARTAMENTO']), axis=1)
df['POSIBLE_DUPLICADO'] = df.groupby('CLAVE_DUP')['CLAVE_DUP'].transform('size').gt(1)

# 7) Guardado
CSV_CLEAN = '/mnt/data/establecimientos_diversificado_limpio.csv'
CSV_DUPS  = '/mnt/data/posibles_duplicados_nuevo.csv'
CSV_LOG   = '/mnt/data/bitacora_limpieza_nueva.csv'
df.to_csv(CSV_CLEAN, index=False, encoding='utf-8')
df.loc[df['POSIBLE_DUPLICADO'], ['CLAVE_DUP','CODIGO','ESTABLECIMIENTO','DIRECCION','TELEFONO','MUNICIPIO','DEPARTAMENTO','JORNADA','PLAN']]\
  .sort_values(['DEPARTAMENTO','MUNICIPIO','ESTABLECIMIENTO','DIRECCION']).to_csv(CSV_DUPS, index=False, encoding='utf-8')
import pandas as pd
pd.DataFrame(bitacora).to_csv(CSV_LOG, index=False, encoding='utf-8')
CSV_CLEAN, CSV_DUPS, CSV_LOG